###IMPORTING NECESSARY MODULES AND PACKAGES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
import re
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.svm import SVC

###TRAINING AND TEST SET

In [ ]:
hasoc_train=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/en_Hasoc2021_train.csv") # hasoc only data
# combined data of hasoc+kaggle
hasoc_test=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/en_Hasoc2021_test_data.csv") # hasoc test data
hasoc_test_bin=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/1A_English_actual_labels.csv") # hasoc binary test labels
hasoc_test_mul=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/1B_English_actual_labels.csv") #hasoc multiclass test labels

In [ ]:
combined_train=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/hate_bin_combine.csv",encoding = "ISO-8859-1") 

In [ ]:
hasoc_train

,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN
...,...,...,...,...,...
3838,1661,60c5d6bf5659ea5e55defd57,@BBCNews Let the dog deal with the wanker once...,HOF,PRFN
3839,194,60c5d6bf5659ea5e55def185,India has suffered a lot. That Chinese bastard...,HOF,HATE
3840,3988,60c5d6bf5659ea5e55def78c,People didn't give 300+ seats majority to BJP ...,HOF,HATE
3841,4212,60c5d6bf5659ea5e55defb04,"@KanganaTeam This is such a vile, xenophobic a...",HOF,PRFN


In [ ]:
combined_train

,_id,text,task_1
0,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF
1,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF
2,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT
3,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF
4,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF
...,...,...,...
35800,31958,ate @user isz that youuu?ÃÂ°ÃÂÃÂÃÂÃÂ°Ã...,NOT
35801,31959,to see nina turner on the airwaves trying to w...,NOT
35802,31960,listening to sad songs on a monday morning otw...,NOT
35803,31961,"@user #sikh #temple vandalised in in #calgary,...",HOF


## binary test data

In [ ]:
hasoc_test["task_1"]=hasoc_test_bin["label"]

## multiclass test data

In [ ]:
hasoc_test["task_2"]=hasoc_test_mul["label"]

In [ ]:
hasoc_test.head()

,_id,text,task_2,task_1
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...,NONE,NOT
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...,PRFN,HOF
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker,PRFN,HOF
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...,NONE,NOT
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ...",NONE,NOT


##PREPROCESSING AND CLEANING
####1.LOWER CASE CONVERTING
####2.REMOVING BAD SYMBOLS
####3.STOP WORD REMOVER
####4.PUNCTUATION REMOVER
####5.TOKENIZATION
####6.LEMMATIZATION
####7.REMOVE USERNAMES 


In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not"])


In [ ]:
def preprocess(tweets):
  
  tweets.encode('ascii', 'ignore').decode('ascii') #remove emojis
  tweets=tweets.lower()                   #convert to lower case
  tweets = re.sub(r"http\S+", " ", tweets) #remove urls
  l=tweets.split(" ")
  for n,i in enumerate(l):
    if '@' in i:
      l[n]=""
  
  tweets=" ".join(l)
  tweets= re.sub('[^A-Za-z]+',' ',tweets) #remove bad character
  tweets = [word for word in tweets.split(" ") if not word in stop_words] # removing stop words
  tweets= [lemmatizer.lemmatize(token, "v") for token in tweets] #Lemmatization
  tweets=" ".join(tweets) 
  return tweets 
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text


## PREPROCESSING THE TRAIN DATA

WHILE USING ONLY HASOC DATA

In [ ]:
hasoc_train["text"]=hasoc_train["text"].apply(lambda x:decontract(x))
hasoc_train["text"]=hasoc_train["text"].apply(lambda x:preprocess(x))

In [ ]:
train_df, val_df = train_test_split(hasoc_train, test_size = 0.2, random_state = 42)
train_tweet=train_df.text
val_tweet=val_df.text

WHILE USING COMBINED DATA

In [ ]:
combined_train["text"]=combined_train["text"].apply(lambda x:decontract(x))
combined_train["text"]=combined_train["text"].apply(lambda x:preprocess(x))

In [ ]:
train_df, val_df = train_test_split(combined_train, test_size = 0.2, random_state = 42)
train_tweet=train_df.text
val_tweet=val_df.text

## PREPROCESSING THE TEST DATA

In [ ]:
hasoc_test["text"]=hasoc_test["text"].apply(lambda x:decontract(x))
hasoc_test["text"]=hasoc_test["text"].apply(lambda x:preprocess(x))

In [ ]:
test_tweet=hasoc_test.text

### on evaluation data

In [ ]:
le=LabelEncoder()

for binary data

In [ ]:
train_df["task_1"] = le.fit_transform(train_df["task_1"])
val_df["task_1"] = le.fit_transform(val_df["task_1"])

for multiclass data

In [ ]:
train_df["task_2"] = le.fit_transform(train_df["task_2"])
val_df["task_2"] = le.fit_transform(val_df["task_2"])

In [ ]:
full_train_tweet=hasoc_train["text"]

In [ ]:
train_tweet=train_df["text"]

In [ ]:
val_tweet=val_df["text"]

### on test data

for binary data

In [ ]:
hasoc_test["task_1"]=le.fit_transform(hasoc_test["task_1"])

for multiclass data

In [ ]:
hasoc_test["task_2"]=le.fit_transform(hasoc_test["task_2"])

##VECTORIZING THE DATA

USING TFIDF VECTORIZATION for ngrams

In [ ]:
vectorizer1 = TfidfVectorizer(ngram_range = (1,1)) #ONLY UNIGRAMS , 
#vectorizer1 = TfidfVectorizer(ngram_range = (2,2)) #ONLY BIGRAM , 
#vectorizer1 = TfidfVectorizer(ngram_range = (1,2)) #UNI + BI , 

full_train_vectors=vectorizer1.fit_transform(full_train_tweet)
train_vectors=vectorizer1.transform(train_tweet)
val_vectors=vectorizer1.transform(val_tweet)

In [ ]:
test_vectors=vectorizer1.transform(test_tweet)

In [ ]:
train_vectors

<3074x8856 sparse matrix of type '<class 'numpy.float64'>'
	with 39804 stored elements in Compressed Sparse Row format>

## for binary

In [ ]:
t1=train_df["task_1"]
t2=val_df["task_1"]

In [ ]:
t3=hasoc_test["task_1"]

## for multiclass

In [ ]:
t4=train_df["task_2"]
t5=val_df["task_2"]
t6=hasoc_test["task_2"]

In [ ]:
from sklearn.metrics import classification_report

##using OPTIMUM KERNEL AND PARAMETERS FOR SVM

In [ ]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

# FOR UNIGRAM ONLY

## FOR BINARY CLASSIFICATION

### PERFORMANCE ON ONLY HASOC DATA 

VALIDATION 80:20 SPLIT

In [ ]:
for i in range(4):
    # Separate data into test and training sets
    svclassifier = getClassifier(i) 
    svclassifier.fit(train_vectors,t1)# Make prediction
    y_pred = svclassifier.predict(val_vectors)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(t2,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       240
           1       0.69      1.00      0.82       529

    accuracy                           0.69       769
   macro avg       0.34      0.50      0.41       769
weighted avg       0.47      0.69      0.56       769



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       240
           1       0.69      1.00      0.82       529

    accuracy                           0.69       769
   macro avg       0.34      0.50      0.41       769
weighted avg       0.47      0.69      0.56       769



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       240
           1       0.69      1.00      0.82       529

    accuracy                           0.69       769
   macro avg       0.34      0.50      0.41       769
weighted avg       0.47      0.69      0.56       769



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.72      0.53      0.61       240
           1       0.81      0.91      0.86       529

    accuracy                           0.79       769
   macro avg       0.76      0.72      0.73       769
weighted avg       0.78      0.79      0.78       769



performance on test data

In [ ]:
for i in range(4):
    # Separate data into test and training sets
    svclassifier = getClassifier(i) 
    svclassifier.fit(train_vectors,t1)# Make prediction
    y_pred = svclassifier.predict(test_vectors)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(t3,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       483
           1       0.62      1.00      0.77       798

    accuracy                           0.62      1281
   macro avg       0.31      0.50      0.38      1281
weighted avg       0.39      0.62      0.48      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       483
           1       0.62      1.00      0.77       798

    accuracy                           0.62      1281
   macro avg       0.31      0.50      0.38      1281
weighted avg       0.39      0.62      0.48      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       483
           1       0.62      1.00      0.77       798

    accuracy                           0.62      1281
   macro avg       0.31      0.50      0.38      1281
weighted avg       0.39      0.62      0.48      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.75      0.57      0.65       483
           1       0.77      0.89      0.82       798

    accuracy                           0.77      1281
   macro avg       0.76      0.73      0.74      1281
weighted avg       0.76      0.77      0.76      1281



### FOR COMBINED (HASOC + KAGGLE DATA)

ON VALIDATION DATA

In [ ]:
for i in range(4):
    # Separate data into test and training sets
    svclassifier = getClassifier(i) 
    svclassifier.fit(train_vectors,t1)# Make prediction
    y_pred = svclassifier.predict(val_vectors)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(t2,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      6216
           1       0.00      0.00      0.00       945

    accuracy                           0.87      7161
   macro avg       0.43      0.50      0.46      7161
weighted avg       0.75      0.87      0.81      7161



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      6216
           1       0.00      0.00      0.00       945

    accuracy                           0.87      7161
   macro avg       0.43      0.50      0.46      7161
weighted avg       0.75      0.87      0.81      7161



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      6216
           1       0.00      0.00      0.00       945

    accuracy                           0.87      7161
   macro avg       0.43      0.50      0.46      7161
weighted avg       0.75      0.87      0.81      7161



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      6216
           1       0.79      0.53      0.64       945

    accuracy                           0.92      7161
   macro avg       0.86      0.75      0.80      7161
weighted avg       0.91      0.92      0.91      7161



ON TEST DATA

In [ ]:
for i in range(4):
    # Separate data into test and training sets
    svclassifier = getClassifier(i) 
    svclassifier.fit(train_vectors,t1)# Make prediction
    y_pred = svclassifier.predict(test_vectors)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(t3,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.38      1.00      0.55       483
           1       0.00      0.00      0.00       798

    accuracy                           0.38      1281
   macro avg       0.19      0.50      0.27      1281
weighted avg       0.14      0.38      0.21      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.38      1.00      0.55       483
           1       0.00      0.00      0.00       798

    accuracy                           0.38      1281
   macro avg       0.19      0.50      0.27      1281
weighted avg       0.14      0.38      0.21      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.38      1.00      0.55       483
           1       0.00      0.00      0.00       798

    accuracy                           0.38      1281
   macro avg       0.19      0.50      0.27      1281
weighted avg       0.14      0.38      0.21      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.59      0.75      0.66       483
           1       0.82      0.68      0.74       798

    accuracy                           0.71      1281
   macro avg       0.70      0.72      0.70      1281
weighted avg       0.73      0.71      0.71      1281



## FOR MULTICLASS CLASSIFICATION

FOR VALIDATION 80:20 SPLIT

In [ ]:
for i in range(4):
    # Separate data into test and training sets
    svclassifier = getClassifier(i) 
    svclassifier.fit(train_vectors,t4)# Make prediction
    y_pred = svclassifier.predict(val_vectors)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(t5,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       141
           1       0.31      1.00      0.48       240
           2       0.00      0.00      0.00       140
           3       0.00      0.00      0.00       248

    accuracy                           0.31       769
   macro avg       0.08      0.25      0.12       769
weighted avg       0.10      0.31      0.15       769



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       141
           1       0.31      1.00      0.48       240
           2       0.00      0.00      0.00       140
           3       0.00      0.00      0.00       248

    accuracy                           0.31       769
   macro avg       0.08      0.25      0.12       769
weighted avg       0.10      0.31      0.15       769



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       141
           1       0.31      1.00      0.48       240
           2       0.00      0.00      0.00       140
           3       0.00      0.00      0.00       248

    accuracy                           0.31       769
   macro avg       0.08      0.25      0.12       769
weighted avg       0.10      0.31      0.15       769



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.50      0.49      0.49       141
           1       0.62      0.70      0.66       240
           2       0.57      0.26      0.35       140
           3       0.73      0.88      0.80       248

    accuracy                           0.64       769
   macro avg       0.61      0.58      0.58       769
weighted avg       0.63      0.64      0.62       769



ON TEST DATA

In [ ]:
for i in range(4):
    # Separate data into test and training sets
    svclassifier = getClassifier(i) 
    svclassifier.fit(train_vectors,t4)# Make prediction
    y_pred = svclassifier.predict(test_vectors)# Evaluate our model
    print("testing:", kernels[i], "kernel")
    print(classification_report(t6,y_pred))

testing: Polynomial kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       224
           1       0.38      1.00      0.55       483
           2       0.00      0.00      0.00       195
           3       0.00      0.00      0.00       379

    accuracy                           0.38      1281
   macro avg       0.09      0.25      0.14      1281
weighted avg       0.14      0.38      0.21      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


testing: RBF kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       224
           1       0.38      1.00      0.55       483
           2       0.00      0.00      0.00       195
           3       0.00      0.00      0.00       379

    accuracy                           0.38      1281
   macro avg       0.09      0.25      0.14      1281
weighted avg       0.14      0.38      0.21      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


testing: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       224
           1       0.38      1.00      0.55       483
           2       0.00      0.00      0.00       195
           3       0.00      0.00      0.00       379

    accuracy                           0.38      1281
   macro avg       0.09      0.25      0.14      1281
weighted avg       0.14      0.38      0.21      1281



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


testing: Linear kernel
              precision    recall  f1-score   support

           0       0.49      0.44      0.47       224
           1       0.66      0.69      0.68       483
           2       0.48      0.22      0.30       195
           3       0.69      0.89      0.78       379

    accuracy                           0.63      1281
   macro avg       0.58      0.56      0.55      1281
weighted avg       0.61      0.63      0.61      1281

